# 1°

In [115]:
import re 
import requests
headers_Get = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1'
    }

Ejemplo de búsqueda:

In [272]:
title='Evidence of Open Access of scientific publications in Google Scholar: a large-scale analysis'# Evidence of Open Access of scientific publications in Google Scholar: a large-scale analysis
author='Alberto Martín-Martín'# Alberto Martín-Martín
coauthors=['R Costas', 'T van Leeuwen']
doi='10.1016/J.JOI.2018.06.012'
year='2006'
publisher='Oxford University Press'
journal=''
volume=''
issue=''
pages=''

TAREA: Implementar
* title,
* cites number
* link al PDF
* Journal 
* Year
* abstract
* Perfil de Google scholar para los autores

In [275]:
def getmetadatos(res):
    gsr={}
    try:
        fr=res.find_all('div',{'class':'gs_r gs_or gs_scl'})[0]
    except IndexError:
        fr=BeautifulSoup('',"html.parser")
        
    gsa=fr.find('div', {'class':"gs_a"}) #¿CÓMO SE RESUELVE ACÁ CON LOS COAUTHORS?
    
    if gsa:
        gsr['authors']=gsa.text.split('\xa0-')[0]
    else:
        gsr['authors']=''
    return gsr

    
def googlescholar(title='', author='', coauthors=[], doi='', year=0, publisher='',
                  journal='', volume='', issue='', pages=0, ) :
    import sys
    gsr={}
    
    s = requests.Session()
    url='https://scholar.google.co.in/scholar_lookup?'
    
    nl=0
    if title:
        nl=nl+1
        url= url+'title={}'.format(title.replace(' ','+'))
    if author:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'author={}'.format(author.replace(' ','+'))
    if doi:
        if nl: url= url+'&'
        nl=nl+1    
        url= url+'doi={}'.format(doi)
    if year:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'year={}'.format(year)
    if publisher:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'publisher={}'.format(publisher.replace(' ','+'))
    if journal:
        if nl: url= url+'&'
        nl=nl+1     
        url= url+'journal={}'.format(journal.replace(' ','+'))
    if volume:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'volume={}'.format(volume)
    if issue:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'issue={}'.format(issue)   
    if pages:
        if nl: url= url+'&'
        nl=nl+1
        url= url+'pages={}'.format(pages)
    if coauthors:
        for i in coauthors :
            if nl: url= url+'&'
            nl=nl+1
            url= url+'coauthors={}'.format(i.replace(' ','+'))        
 
            
    r=s.get(url,headers=headers_Get)
    #print(url)
       
    
    # interpretar HTML
    from bs4 import BeautifulSoup
    res=BeautifulSoup(r.text,"html.parser")
   
    gsr=getmetadatos(res)
        
    return gsr 

In [276]:
googlescholar(title='Evidence of Open Access of scientific publications in Google Scholar: a large-scale analysis',
            author='Alberto Martín-Martín',
            coauthors=['R Costas', 'T van Leeuwen'] )

https://scholar.google.co.in/scholar_lookup?title=Evidence+of+Open+Access+of+scientific+publications+in+Google+Scholar:+a+large-scale+analysis&author=Alberto+Martín-Martín&coauthors=R+Costas&coauthors=T+van+Leeuwen


{'authors': 'A Martín-Martín, R Costas, T van Leeuwen…'}

In [ ]:
title='Evidence of Open Access of scientific publications in Google Scholar: a large-scale analysis'# Evidence of Open Access of scientific publications in Google Scholar: a large-scale analysis
author='Alberto Martín-Martín'# Alberto Martín-Martín
coauthors=[''] #'
doi='' # 10.1016/J.JOI.2018.06.012
year=''
publisher='' #Oxford University Press
journal=''
volume=''
issue=''
pages=''
hl=''

In [ ]:
title='', author='', coauthors=[''], doi='', year='', publisher='', journal='', volume='', issue='', pages='', hl=''

# Temnporal
res → record

In [ ]:
def get_google_scholar(record):
    '''
    Analyise the BeautifulSoup record for an article 
    in Google Scholar.
    Output is a Python dictionary with keys: 
    'title', 'authors','profiles','Jornal','Year',
    'abstract','cites','cites_link'
    '''
    import random
    import time
    gsr={}
    try:
        cites=record.find_all('a',{"href":re.compile( "/scholar\?cites*" )})[0]
        try:
            gsr['cites']=int( cites.text.split()[-1] )
            gsr['cites_link']=cites.attrs.get('href')
        except:
            gsr['cites']=0
    except:
        cites=None

    # Title
    try:
        #The .split('XXX')[-1]  does not afect the result when .text does no contains 'XXX'
        tc=record.find_all('h3',{"class":"gs_rt"})[0].text.split('[CITATION][C] ')[-1]
    except:
        tc=''

    gsr['title']=tc.strip().split('[HTML][HTML] ')[-1].split(
                                  '[PDF][PDF] '  )[-1]
    
    # Explore authors, google scholar profile, Journal and Year
    gpa=None
    try:
        gpa=record.find_all('div',{"class":"gs_a"})[0]
        gsr['ref']=gpa.text.strip()
        gsr['authors']=gpa.text.split('-')[0].strip()
        try:
            jy=gpa.text.split('-')[1].strip()
            gsr['Journal']=jy.split(',')[0]
            try:
                gsr['Year']=eval(jy.split(',')[1])
            except:
                gsr['Year']=-1
        except:
            gsr['Journal']=''
    except:
        gsr['authors']=''
        gsr['ref']=''

    #Abstract:
    try:
        gsr['abstract']=record.find_all('div',{'class':'gs_rs'})[0].text.replace('\xa0…','')
    except:
        gsr['abstract']=''
    # citations
    if gpa:
        lpr=gpa.find_all("a",{ "href":re.compile("/citations\?user=*")   } )
        prf={}
        for pr in lpr:
            prf[ pr.text ]=pr.attrs.get('href').split('?')[-1].split('&')[0].split('user=')[-1]
        gsr['profiles']=prf
    
    time.sleep( random.randint(1,3)  ) # avoid robots
    return gsr